In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

epoch_count = 30

In [ ]:
gpu_options = tf.GPUOptions(allow_growth=True)
def get_sigma_by_snr(snr, ave_power, signal_l):
    sigma2 = ave_power / (signal_l * np.power(10.0, snr/10.0))
    sigma = np.sqrt(sigma2)
    return sigma

def get_final_result_for_detector(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.5:
            r[i] = 0
    return r

def get_final_result_for_decoder(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.05:
            r[i] = 0
    return r

def get_matrix(size, length):
    ma = 1-2*np.random.binomial(1,0.5, (size,length))
    mb = np.random.binomial(1,0.5, (size,length))
    return ma*mb

def get_batch(data_size, p, phi, total):
    phi = np.transpose(phi)
    x = np.random.binomial(n=1, p=p, size=(total,data_size))
    h_x = np.random.rayleigh(1, size=(total,data_size)) * x
    phi_h_x = np.dot(phi, h_x)
    phi_x = np.dot(phi, x)
    return np.transpose(phi_x), np.transpose(x), np.transpose(phi_h_x), np.transpose(h_x)

def get_avege_power(p, phi, total, test_time):
    phi = np.transpose(phi)
    sum=0
    for i in range(test_time):
        x = np.random.binomial(n=1, p=p, size=(total,1))
        phi_x = np.dot(phi, x)
        sum += np.power(np.linalg.norm(phi_x, ord=2), 2)
    return sum/test_time

In [ ]:
l = 50
total = 100
batch_size = 1000
p = 0.1
phi = np.array(get_matrix(total, l), 'float32')
ave_power = get_avege_power(p,phi,total,100000)
# Training Parameters
learning_rate = 0.001
display_step = 1000
train_sigma = get_sigma_by_snr(10, ave_power, l)
print(ave_power)
# Network Parameters
num_input = l # input of decoder
num_hidden_decoder = (l + total)# decoder layer num features
num_output = total # output of decoder or this NN

In [ ]:
# tf input
X1 = tf.placeholder("float", [None, num_input])
X2 = tf.placeholder("float", [None, num_output])

dc_label = tf.placeholder("float", [None, num_output])
dt_label = tf.placeholder("float", [None, num_output])

is_training = tf.placeholder_with_default(False,shape=[])

g_noise = tf.placeholder("float", [None, num_input])

weights = {
    'decoder_h0': tf.get_variable("decoder_h0", shape=[num_input, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h1': tf.get_variable("decoder_h1", shape=[num_input + num_output, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h2': tf.get_variable("decoder_h2", shape=[num_hidden_decoder, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h3': tf.get_variable("decoder_h3", shape=[num_hidden_decoder, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h4': tf.get_variable("decoder_h4", shape=[num_hidden_decoder, num_output], initializer=tf.contrib.layers.xavier_initializer()),
}

biases = {
    'decoder_b0': tf.get_variable("decoder_b0", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b1': tf.get_variable("decoder_b1", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b2': tf.get_variable("decoder_b2", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b3': tf.get_variable("decoder_b3", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()), 
    'decoder_b4': tf.get_variable("decoder_b4", shape=[num_output], initializer=tf.contrib.layers.xavier_initializer()),
}

weights_detector = {
    'decoder_h0': tf.get_variable("decoder_h0_dt", shape=[num_input, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h1': tf.get_variable("decoder_h1_dt", shape=[num_input + num_output, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h2': tf.get_variable("decoder_h2_dt", shape=[num_hidden_decoder, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h3': tf.get_variable("decoder_h3_dt", shape=[num_hidden_decoder, num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h4': tf.get_variable("decoder_h4_dt", shape=[num_hidden_decoder, num_output], initializer=tf.contrib.layers.xavier_initializer()),
}

biases_detector = {
    'decoder_b0': tf.get_variable("decoder_b0_dt", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b1': tf.get_variable("decoder_b1_dt", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b2': tf.get_variable("decoder_b2_dt", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_b3': tf.get_variable("decoder_b3_dt", shape=[num_hidden_decoder], initializer=tf.contrib.layers.xavier_initializer()), 
    'decoder_b4': tf.get_variable("decoder_b4_dt", shape=[num_output], initializer=tf.contrib.layers.xavier_initializer()),
}

In [ ]:
# 构建decoder
def estimator_0(x):
    bn_input = tf.layers.batch_normalization(x, training=is_training)
    layer_1 = tf.nn.relu(tf.add(tf.matmul(bn_input, weights['decoder_h0']),
                                biases['decoder_b0']))
    return layer_1
    
def detector_0(x):
    bn_input = tf.layers.batch_normalization(x, training=is_training)
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(bn_input, weights_detector['decoder_h0']),
                                biases_detector['decoder_b0']))
    return layer_1

def g1_estimator(x):
    bn_input = tf.layers.batch_normalization(x, training=is_training)
    layer_1 = tf.nn.relu(tf.add(tf.matmul(bn_input, weights['decoder_h1']),
                                biases['decoder_b1']))
    bn_layer_1 = tf.layers.batch_normalization(layer_1, training=is_training)
    layer_2 = tf.nn.relu(tf.add(tf.matmul(bn_layer_1, weights['decoder_h2']),
                                biases['decoder_b2']))
    bn_layer_2 = tf.layers.batch_normalization(layer_2, training=is_training)
    layer_3 = tf.nn.relu(tf.add(tf.matmul(bn_layer_2, weights['decoder_h3']),
                                biases['decoder_b3']))
    bn_layer_3 = tf.layers.batch_normalization(layer_3, training=is_training)
    layer_4 = tf.nn.relu(tf.add(tf.matmul(bn_layer_3, weights['decoder_h4']),
                                biases['decoder_b4']))
    return layer_4

def g1_detector(x):
    bn_input = tf.layers.batch_normalization(x, training=is_training)
    layer_1 = tf.nn.relu(tf.add(tf.matmul(bn_input, weights_detector['decoder_h1']),
                                biases_detector['decoder_b1']))
    bn_layer_1 = tf.layers.batch_normalization(layer_1, training=is_training)
    layer_2 = tf.nn.relu(tf.add(tf.matmul(bn_layer_1, weights_detector['decoder_h2']),
                                biases_detector['decoder_b2']))
    bn_layer_2 = tf.layers.batch_normalization(layer_2, training=is_training)
    layer_3 = tf.nn.relu(tf.add(tf.matmul(bn_layer_2, weights_detector['decoder_h3']),
                                biases_detector['decoder_b3']))
    bn_layer_3 = tf.layers.batch_normalization(layer_3, training=is_training)
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(bn_layer_3, weights_detector['decoder_h4']),
                                biases_detector['decoder_b4']))
    return layer_4


In [ ]:
# 构建模型
p_1_p = tf.cond(tf.equal(is_training, tf.constant(True)), lambda:tf.ones([batch_size, num_output]), lambda:[tf.ones([1, num_output])])
p_2_p = tf.cond(tf.equal(is_training, tf.constant(True)), lambda:tf.zeros([batch_size, num_output]), lambda:[tf.zeros([1, num_output])])

mixer_input_d = estimator_0(X1 + g_noise)
mixer_input_e = detector_0(X1 + g_noise)

# with tf.variable_scope("One_Detector"):
g1_detector_op = g1_detector(mixer_input_d)
# with tf.variable_scope("One"):
g1_estimator_op = g1_estimator(mixer_input_e)
    
# g1_s_decoder_op = tf.multiply(g1_decoder_op, g1_detector_op)
# p_2 = (X1 + g_noise) - tf.matmul(g1_s_decoder_op, phi)
g2_mixer_input_d = tf.concat([g1_estimator_op, X1 + g_noise], 1)
g2_mixer_input_e = tf.concat([g1_detector_op, X1 + g_noise], 1)

# with tf.variable_scope("Two"):
g2_estimator_op = g1_estimator(g2_mixer_input_e)
# with tf.variable_scope("Two_Detector"):
g2_detector_op = g1_detector(g2_mixer_input_d)

g3_mixer_input_e = tf.concat([g2_detector_op, X1 + g_noise], 1)
g3_mixer_input_d = tf.concat([g2_estimator_op, X1 + g_noise], 1)

# with tf.variable_scope("Three"):
g3_estimator_op = g1_estimator(g3_mixer_input_e)
# with tf.variable_scope("Three_Detector"):
g3_detector_op = g1_detector(g3_mixer_input_d)


g4_mixer_input_e = tf.concat([g3_detector_op, X1 + g_noise], 1)
g4_mixer_input_d = tf.concat([g3_estimator_op, X1 + g_noise], 1)

# with tf.variable_scope("Four"):
g4_estimator_op = g1_estimator(g4_mixer_input_e)
# with tf.variable_scope("Four_Detector"):
g4_detector_op = g1_detector(g4_mixer_input_d)


g5_mixer_input_e = tf.concat([g4_detector_op, X1 + g_noise], 1)
g5_mixer_input_d = tf.concat([g4_estimator_op, X1 + g_noise], 1)

# with tf.variable_scope("Five"):
g5_estimator_op = g1_estimator(g5_mixer_input_e)
# with tf.variable_scope("Five_Detector"):
g5_detector_op = g1_detector(g5_mixer_input_d)



# #6
# p_6 = (X1 + g_noise) - tf.matmul(g5_s_decoder_op, phi)
# g6_mixer_input = tf.concat([g5_s_decoder_op, p_6], 1)

# # with tf.variable_scope("Five"):
# g6_decoder_op = g1_decoder(g6_mixer_input)

# # with tf.variable_scope("Five_Detector"):
# g6_detector_op = g1_detector(g6_mixer_input)

# g6_s_decoder_op = tf.multiply(g6_decoder_op, g6_detector_op)

# #7
# p_7 = (X1 + g_noise) - tf.matmul(g6_s_decoder_op, phi)
# g7_mixer_input = tf.concat([g6_s_decoder_op, p_7], 1)

# # with tf.variable_scope("Five"):
# g7_decoder_op = g1_decoder(g7_mixer_input)

# # with tf.variable_scope("Five_Detector"):
# g7_detector_op = g1_detector(g7_mixer_input)

# g7_s_decoder_op = tf.multiply(g7_decoder_op, g7_detector_op)

# #8
# p_8 = (X1 + g_noise) - tf.matmul(g7_s_decoder_op, phi)
# g8_mixer_input = tf.concat([g7_s_decoder_op, p_8], 1)

# # with tf.variable_scope("Five"):
# g8_decoder_op = g1_decoder(g8_mixer_input)

# # with tf.variable_scope("Five_Detector"):
# g8_detector_op = g1_detector(g8_mixer_input)

# g8_s_decoder_op = tf.multiply(g8_decoder_op, g8_detector_op)

In [ ]:
# decoder_op = g3_decoder_op
# detector_op = g3_detector_op

# 预测
# y_pred_dc = decoder_op
# y_pred_dt = detector_op
y_true_dc = dc_label
y_true_dt = dt_label
 
# loss函数和优化器
g1_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g1_estimator_op)
g1_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g1_detector_op)

g2_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g2_estimator_op)
g2_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g2_detector_op)


g3_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g3_estimator_op)
g3_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g3_detector_op)

g4_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g4_estimator_op)
g4_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g4_detector_op)

g5_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g5_estimator_op)
g5_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g5_detector_op)

# g6_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g6_decoder_op)
# g6_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g6_detector_op)

# g7_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g7_decoder_op)
# g7_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g7_detector_op)

# g8_mse_l_dc = tf.losses.mean_squared_error(y_true_dc, g8_decoder_op)
# g8_mse_l_dt = tf.losses.mean_squared_error(y_true_dt, g8_detector_op)


mul_mse_l_dc = g1_mse_l_dc + g2_mse_l_dc + g3_mse_l_dc + g4_mse_l_dc + g5_mse_l_dc
mul_mse_l_dt = g1_mse_l_dt + g2_mse_l_dt + g3_mse_l_dt + g4_mse_l_dt + g5_mse_l_dt

# mul_mse_l_dc = g1_mse_l_dc + g2_mse_l_dc + g3_mse_l_dc + g4_mse_l_dc + g5_mse_l_dc + g6_mse_l_dc + g7_mse_l_dc + g8_mse_l_dc
# mul_mse_l_dt = g1_mse_l_dt + g2_mse_l_dt + g3_mse_l_dt + g4_mse_l_dt + g5_mse_l_dt + g6_mse_l_dt + g7_mse_l_dt + g8_mse_l_dt

mul_mse_l = mul_mse_l_dc + mul_mse_l_dt

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

mul_optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(mul_mse_l)



init = tf.global_variables_initializer()

In [ ]:
# Start Training
# Start a new TF session
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# Run the initializer
sess.run(init)

ini_weights = sess.run(weights)

ini_biases = sess.run(biases)

now_step = 0
now_step_dt = 0

final_loss = 0

cost_list = []
cost_list_1=[]
cost_list_2=[]
cost_list_3=[]
cost_list_4=[]
cost_list_5=[]
cost_list_dt = []
cost_list_dt_1 = []
cost_list_dt_2 = []
cost_list_dt_3 = []
cost_list_dt_4 = []
cost_list_dt_5 = []
step_list = []
step_list_for_test = []
result_list = []
step_list_dt = []

In [ ]:
for m in range (epoch_count*5):
#     for i in range(int(1000000 / batch_size)):
    for i in range(500):
        phi_x, x, phi_h_x, h_x = get_batch(batch_size, p, phi, total)
        _, _, l_dc, l_dc_1, l_dc_2, l_dc_3, l_dc_4, l_dc_5, l_dt, l_dt_1, l_dt_2, l_dt_3, l_dt_4, l_dt_5 = sess.run([update_ops, mul_optimizer, mul_mse_l_dc, g1_mse_l_dc, g2_mse_l_dc, g3_mse_l_dc, g4_mse_l_dc, g5_mse_l_dc, mul_mse_l_dt, g1_mse_l_dt, g2_mse_l_dt, g3_mse_l_dt, g4_mse_l_dt, g5_mse_l_dt], feed_dict={X1: phi_h_x, dc_label:h_x,
                                                             X2:h_x, dt_label: x, g_noise: np.random.normal(loc=0, scale=train_sigma, size=(batch_size, num_input)), is_training: True})
        now_step += 1
        # Display logs per step
        if i % display_step == 0:
            now = time.ctime()
            cnvtime = time.strptime(now)
            print(time.strftime("%Y/%m/%d %H:%M", cnvtime))
            print('Step %i: Minibatch Loss: %f\n %f\t%f\t%f\t%f\t%f' % (now_step, l_dc, l_dc_1, l_dc_2, l_dc_3, l_dc_4, l_dc_5))
            print('%f\n %f\t%f\t%f\t%f\t%f' % (l_dt, l_dt_1, l_dt_2, l_dt_3, l_dt_4, l_dt_5))
            step_list.append(now_step)
            cost_list.append(l_dc)
            cost_list_1.append(l_dc_1)
            cost_list_2.append(l_dc_2)
            cost_list_3.append(l_dc_3)
            cost_list_4.append(l_dc_4)
            cost_list_5.append(l_dc_5)
            cost_list_dt.append(l_dt)
            cost_list_dt_1.append(l_dt_1)
            cost_list_dt_2.append(l_dt_2)
            cost_list_dt_3.append(l_dt_3)
            cost_list_dt_4.append(l_dt_4)
            cost_list_dt_5.append(l_dt_5)
        final_loss = l_dc
now = time.ctime()
cnvtime = time.strptime(now)
print(time.strftime("%Y/%m/%d %H:%M", cnvtime))
print('This training is end, and final cost is %f\t%f' % (final_loss,l_dt))

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
plt.plot(p1_x, cost_list_dt, label='detector')
plt.plot(p1_x, p1_y, label='estimator')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('T_C_2.eps')
plt.show()

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
# plt.plot(p1_x, cost_list_1, label='1')
# plt.plot(p1_x, cost_list_2, label='2')
# plt.plot(p1_x, cost_list_3, label='3')
# plt.plot(p1_x, cost_list_4, label='4')
plt.plot(p1_x, cost_list_dt_5, label='5_d')
plt.plot(p1_x, cost_list_5, label='5_e')

# plt.plot(p1_x, cost_list_dt, label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
plt.show()

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
plt.plot(p1_x, cost_list_1, label='1')
plt.plot(p1_x, cost_list_2, label='2')
plt.plot(p1_x, cost_list_3, label='3')
plt.plot(p1_x, cost_list_4, label='4')
# plt.plot(p1_x, cost_list_dt_5, label='5_d')
plt.plot(p1_x, cost_list_5, label='5_e')

# plt.plot(p1_x, cost_list_dt, label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
plt.show()

In [ ]:
ave_power

In [ ]:
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 1000
skip_time = 0
phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
output_x_o = np.ones_like(h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
    if (np.linalg.norm(x, 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    one_one = get_final_result_for_decoder(output)
    output_x_o = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    error_bits += np.linalg.norm(x - one_one,1)
    if error_bits != 0:
        error_blocks += 1
    eve += (np.power(np.linalg.norm(output[0]-h_x, 2), 2) / np.power(np.linalg.norm(h_x, 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)
print(error_bits/((loop_time-skip_time)*total))

In [ ]:
v_sigma = get_sigma_by_snr(20, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 1000
skip_time = 0
phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
output_x_o = np.ones_like(h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
    if (np.linalg.norm(x, 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    one_one = get_final_result_for_decoder(output)
    output_x_o = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    error_bits += np.linalg.norm(x - one_one,1)
    if error_bits != 0:
        error_blocks += 1
    eve += (np.power(np.linalg.norm(output[0]-h_x, 2), 2) / np.power(np.linalg.norm(h_x, 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)
print(error_bits/((loop_time-skip_time)*total))

In [ ]:
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
output_x_o = np.ones_like(h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
    if (np.linalg.norm(x, 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    one_one = get_final_result_for_decoder(output)
    output_x_o = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    error_bits += np.linalg.norm(x - one_one,1)
    if error_bits != 0:
        error_blocks += 1
    eve += (np.power(np.linalg.norm(output[0]-h_x, 2), 2) / np.power(np.linalg.norm(h_x, 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)
print(error_bits/((loop_time-skip_time)*total))

In [ ]:
v_sigma = get_sigma_by_snr(30, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 1000
skip_time = 0
phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
output_x_o = np.ones_like(h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
    if (np.linalg.norm(x, 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output_x_o = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-h_x, 2), 2) / np.power(np.linalg.norm(h_x, 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
v_sigma = get_sigma_by_snr(20, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
output_x_o = np.ones_like(h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    phi_x, x, phi_h_x, h_x = get_batch(1, p, phi, total)
    if (np.linalg.norm(x, 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: phi_h_x,
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output_x_o = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-h_x, 2), 2) / np.power(np.linalg.norm(h_x, 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
ave_power

In [ ]:
h_x

In [ ]:
detector_output = n_phi_h_x
bbb= n_x
aaa = n_h_x
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = n_phi_h_x
bbb= n_x
aaa = n_h_x
v_sigma = get_sigma_by_snr(40, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 10
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output = [output[0] * get_final_result_for_decoder(output)]
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = n_phi_h_x
bbb= n_x
aaa = n_h_x
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 10000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output = [output[0] * get_final_result_for_decoder(output)]
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('BPSK_50_100_1V_Rando_phi_h_x.npy')
aaa = np.load('BPSK_50_100_1V_Rando_h_x.npy')
bbb= np.load('BPSK_50_100_1V_Rando_x.npy')
ave_power = np.load("BPSK_50_100_1V_Rando_ave.npy")
v_sigma = 0.7071067811865476
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 10000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output = [output[0] * get_final_result_for_decoder(output)]
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('BPSK_50_100_1V_Rando_phi_h_x.npy')
aaa = np.load('BPSK_50_100_1V_Rando_h_x.npy')
bbb= np.load('BPSK_50_100_1V_Rando_x.npy')
ave_power = np.load("BPSK_50_100_1V_Rando_ave.npy")
v_sigma = 0.7071067811865476
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 10000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
#     output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
#                                                g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
#                                                is_training: False})
#     output = output * output_2
#     output = [output[0] * get_final_result_for_decoder(output)]
    output_x_o[i] = output
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
v_sigma

In [ ]:
0.7233742323306795

In [ ]:
np.load("BPSK_50_100_1V_Rando_ave.npy")

In [ ]:
v_sigma = get_sigma_by_snr(10, 250, l)

In [ ]:
v_sigma

In [ ]:
ave_power = np.load("BPSK_50_100_1V_Rando_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)

In [ ]:
v_sigma

In [ ]:
detector_output = n_phi_h_x
bbb= n_x
aaa = n_h_x
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = n_phi_h_x
bbb= n_x
aaa = n_h_x
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

#     output = sess.run(g5_estimator_op, feed_dict={X1: [detector_output[i]],
#                                                g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
#                                                is_training: False})
    output_2 = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output = output * output_2
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
bbb[0]-get_final_result_for_decoder(output)

In [ ]:
bbb[0]-get_final_result_for_detector(output_2)

In [ ]:
get_final_result_for_detector(output_2)

In [ ]:
detector_output = np.load('250_500_01_R_phi_h_x.npy')
aaa = np.load('250_500_01_R_h_x.npy')

bbb= np.load('250_500_01_R_x.npy')
ave_power = np.load("250_500_01_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_01_R_phi_h_x.npy')
aaa = np.load('250_500_01_R_h_x.npy')

bbb= np.load('250_500_01_R_x.npy')
ave_power = np.load("250_500_01_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g3_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_1_R_phi_h_x.npy')
aaa = np.load('250_500_1_R_h_x.npy')
bbb= np.load('250_500_1_R_x.npy')
ave_power = np.load("250_500_1_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_05_R_phi_h_x.npy')
aaa = np.load('250_500_05_R_h_x.npy')
bbb= np.load('250_500_05_R_x.npy')
ave_power = np.load("250_500_05_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_01_R_phi_h_x.npy')
aaa = np.load('250_500_01_R_h_x.npy')
bbb= np.load('250_500_01_R_x.npy')
ave_power = np.load("250_500_01_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g3_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_02_R_phi_h_x.npy')
aaa = np.load('250_500_02_R_h_x.npy')
bbb= np.load('250_500_02_R_x.npy')
ave_power = np.load("250_500_02_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('250_500_03_R_phi_h_x.npy')
aaa = np.load('250_500_03_R_h_x.npy')
bbb= np.load('250_500_03_R_x.npy')
ave_power = np.load("250_500_03_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
1.0117705888070674

In [ ]:
v_sigma

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
plt.plot(p1_x, cost_list_dt, label='detector')
plt.plot(p1_x, p1_y, label='estimator')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('T_C_2.eps')
plt.show()

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
# plt.plot(p1_x, cost_list_1, label='1')
# plt.plot(p1_x, cost_list_2, label='2')
# plt.plot(p1_x, cost_list_3, label='3')
# plt.plot(p1_x, cost_list_4, label='4')
plt.plot(p1_x, cost_list_dt_5, label='5_d')
plt.plot(p1_x, cost_list_5, label='5_e')

# plt.plot(p1_x, cost_list_dt, label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
plt.show()

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = np.array(cost_list) + np.array(cost_list_dt)
# plt.plot(p1_x, cost_list_dt, label='detector')
plt.plot(p1_x, p2_y, label='29 X 16')
plt.plot(p1_x, p1_y, label='153 X 64')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
plt.savefig('T_C_3.eps')
# plt.show()

In [ ]:
p2_y = np.load('29X16TS.npy')

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
# plt.plot(p1_x, cost_list_1, label='1')
# plt.plot(p1_x, cost_list_2, label='2')
# plt.plot(p1_x, cost_list_3, label='3')
# plt.plot(p1_x, cost_list_4, label='4')
plt.plot(p1_x, cost_list_5, label='5')
plt.plot(p1_x, cost_list_dt_5, label='5_d')
# plt.plot(p1_x, cost_list_dt, label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
plt.show()

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
plt.plot(p1_x, cost_list_dt_1, label='$MSE_1^d$')
plt.plot(p1_x, cost_list_dt_2, label='$MSE_2^d$')
plt.plot(p1_x, cost_list_dt_3, label='$MSE_3^d$')
plt.plot(p1_x, cost_list_dt_4, label='$MSE_4^d$')
plt.plot(p1_x, cost_list_dt_5, label='$MSE_5^d$')
# plt.plot(p1_x[-10:], cost_list_dt[-10:], label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 27500)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
# plt.show()
plt.savefig('T_T_1.eps')

In [ ]:
# p1_x = step_list[-40:]
# p1_y = cost_list[-40:]
p1_x = step_list
p1_y = cost_list
# plt.plot(p1_x, p1_y, label='decoder_sum')
plt.plot(p1_x, cost_list_1, label='$MSE_1^e$')
plt.plot(p1_x, cost_list_2, label='$MSE_2^e$')
plt.plot(p1_x, cost_list_3, label='$MSE_3^e$')
plt.plot(p1_x, cost_list_4, label='$MSE_4^e$')
plt.plot(p1_x, cost_list_5, label='$MSE_5^e$')
# plt.plot(p1_x[-10:], cost_list_dt[-10:], label='detector')
plt.grid()
plt.yscale('log')
plt.xlabel("batches")
plt.ylabel("loss")
# plt.xlim(right = 76000)
plt.legend()
# plt.savefig('dnn_mixer＿X1_AD_DTO_K_R_LLL_R_L.eps')
# plt.show()
plt.savefig('T_T_2.eps')

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
del detector_output
del bbb
del aaa

In [ ]:
detector_output = np.load('64_4_153_02_R_phi_h_x.npy')
bbb= np.load('64_4_153_02_R_x.npy')
aaa = np.load('64_4_153_02_R_h_x.npy')
ave_power = np.load("64_4_153_02_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_02 = 10 * np.log10(eve)
print(r_02)

In [ ]:
detector_output = np.load('64_4_153_03_R_phi_h_x.npy')
bbb= np.load('64_4_153_03_R_x.npy')
aaa = np.load('64_4_153_03_R_h_x.npy')
ave_power = np.load("64_4_153_03_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_03 = 10 * np.log10(eve)
print(r_03)

In [ ]:
detector_output = np.load('64_4_153_04_R_phi_h_x.npy')
bbb= np.load('64_4_153_04_R_x.npy')
aaa = np.load('64_4_153_04_R_h_x.npy')
ave_power = np.load("64_4_153_04_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_04 = 10 * np.log10(eve)
print(r_04)

In [ ]:
detector_output = np.load('64_4_153_05_R_phi_h_x.npy')
bbb= np.load('64_4_153_05_R_x.npy')
aaa = np.load('64_4_153_05_R_h_x.npy')
ave_power = np.load("64_4_153_05_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_05 = 10 * np.log10(eve)
print(r_05)

In [ ]:
detector_output = np.load('64_4_153_06_R_phi_h_x.npy')
bbb= np.load('64_4_153_06_R_x.npy')
aaa = np.load('64_4_153_06_R_h_x.npy')
ave_power = np.load("64_4_153_06_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_06 = 10 * np.log10(eve)
print(r_06)

In [ ]:
detector_output = np.load('64_4_153_07_R_phi_h_x.npy')
bbb= np.load('64_4_153_07_R_x.npy')
aaa = np.load('64_4_153_07_R_h_x.npy')
ave_power = np.load("64_4_153_07_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_07 = 10 * np.log10(eve)
print(r_07)

In [ ]:
detector_output = np.load('64_4_153_08_R_phi_h_x.npy')
bbb= np.load('64_4_153_08_R_x.npy')
aaa = np.load('64_4_153_08_R_h_x.npy')
ave_power = np.load("64_4_153_08_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_08 = 10 * np.log10(eve)
print(r_08)

In [ ]:
detector_output = np.load('64_4_153_09_R_phi_h_x.npy')
bbb= np.load('64_4_153_09_R_x.npy')
aaa = np.load('64_4_153_09_R_h_x.npy')
ave_power = np.load("64_4_153_09_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_09 = 10 * np.log10(eve)
print(r_09)

In [ ]:
detector_output = np.load('64_4_153_1_R_phi_h_x.npy')
bbb= np.load('64_4_153_1_R_x.npy')
aaa = np.load('64_4_153_1_R_h_x.npy')
ave_power = np.load("64_4_153_1_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_10 = 10 * np.log10(eve)
print(r_10)

In [ ]:
detector_output = np.load('64_4_153_09_R_phi_h_x.npy')
bbb= np.load('64_4_153_09_R_x.npy')
aaa = np.load('64_4_153_09_R_h_x.npy')
ave_power = np.load("64_4_153_09_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    result = get_final_result_for_decoder(output)
    output[0] = output[0]*result
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_09_t = 10 * np.log10(eve)
print(r_09_t)

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01_t = 10 * np.log10(eve)
print(r_01_t)

In [ ]:
detector_output = np.load('64_4_153_02_R_phi_h_x.npy')
bbb= np.load('64_4_153_02_R_x.npy')
aaa = np.load('64_4_153_02_R_h_x.npy')
ave_power = np.load("64_4_153_02_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_02_t = 10 * np.log10(eve)
print(r_02_t)

In [ ]:
detector_output = np.load('64_4_153_03_R_phi_h_x.npy')
bbb= np.load('64_4_153_03_R_x.npy')
aaa = np.load('64_4_153_03_R_h_x.npy')
ave_power = np.load("64_4_153_03_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_03_t = 10 * np.log10(eve)
print(r_03_t)

In [ ]:
detector_output = np.load('64_4_153_04_R_phi_h_x.npy')
bbb= np.load('64_4_153_04_R_x.npy')
aaa = np.load('64_4_153_04_R_h_x.npy')
ave_power = np.load("64_4_153_04_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_04_t = 10 * np.log10(eve)
print(r_04_t)

In [ ]:
detector_output = np.load('64_4_153_05_R_phi_h_x.npy')
bbb= np.load('64_4_153_05_R_x.npy')
aaa = np.load('64_4_153_05_R_h_x.npy')
ave_power = np.load("64_4_153_05_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_05_t = 10 * np.log10(eve)
print(r_05_t)

In [ ]:
detector_output = np.load('64_4_153_06_R_phi_h_x.npy')
bbb= np.load('64_4_153_06_R_x.npy')
aaa = np.load('64_4_153_06_R_h_x.npy')
ave_power = np.load("64_4_153_06_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_06_t = 10 * np.log10(eve)
print(r_06_t)

In [ ]:
detector_output = np.load('64_4_153_07_R_phi_h_x.npy')
bbb= np.load('64_4_153_07_R_x.npy')
aaa = np.load('64_4_153_07_R_h_x.npy')
ave_power = np.load("64_4_153_07_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_07_t = 10 * np.log10(eve)
print(r_07_t)

In [ ]:
detector_output = np.load('64_4_153_08_R_phi_h_x.npy')
bbb= np.load('64_4_153_08_R_x.npy')
aaa = np.load('64_4_153_08_R_h_x.npy')
ave_power = np.load("64_4_153_08_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_08_t = 10 * np.log10(eve)
print(r_08_t)

In [ ]:
detector_output = np.load('64_4_153_1_R_phi_h_x.npy')
bbb= np.load('64_4_153_1_R_x.npy')
aaa = np.load('64_4_153_1_R_h_x.npy')
ave_power = np.load("64_4_153_1_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_10_t = 10 * np.log10(eve)
print(r_10_t)

In [ ]:
result = get_final_result_for_decoder(output)

In [ ]:
[r_01,r_02,r_03,r_04,r_05,r_06,r_07]

In [ ]:
[r_01,r_02,r_03,r_04,r_05,r_06,r_07]

In [ ]:
del detector_output
del bbb
del aaa

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 1000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
print(error_bits/(total * (loop_time-skip_time)))
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
print(error_bits/(total * (loop_time-skip_time)))
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_02_R_phi_h_x.npy')
bbb= np.load('64_4_153_02_R_x.npy')
aaa = np.load('64_4_153_02_R_h_x.npy')
ave_power = np.load("64_4_153_02_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_02 = error_bits/(total * (loop_time-skip_time))
print(BER_D_02)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_03_R_phi_h_x.npy')
bbb= np.load('64_4_153_03_R_x.npy')
aaa = np.load('64_4_153_03_R_h_x.npy')
ave_power = np.load("64_4_153_03_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_03 = error_bits/(total * (loop_time-skip_time))
print(BER_D_03)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_04_R_phi_h_x.npy')
bbb= np.load('64_4_153_04_R_x.npy')
aaa = np.load('64_4_153_04_R_h_x.npy')
ave_power = np.load("64_4_153_04_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_04 = error_bits/(total * (loop_time-skip_time))
print(BER_D_04)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_05_R_phi_h_x.npy')
bbb= np.load('64_4_153_05_R_x.npy')
aaa = np.load('64_4_153_05_R_h_x.npy')
ave_power = np.load("64_4_153_05_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_05 = error_bits/(total * (loop_time-skip_time))
print(BER_D_05)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_06_R_phi_h_x.npy')
bbb= np.load('64_4_153_06_R_x.npy')
aaa = np.load('64_4_153_06_R_h_x.npy')
ave_power = np.load("64_4_153_06_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_06 = error_bits/(total * (loop_time-skip_time))
print(BER_D_06)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_07_R_phi_h_x.npy')
bbb= np.load('64_4_153_07_R_x.npy')
aaa = np.load('64_4_153_07_R_h_x.npy')
ave_power = np.load("64_4_153_07_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_07 = error_bits/(total * (loop_time-skip_time))
print(BER_D_07)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_08_R_phi_h_x.npy')
bbb= np.load('64_4_153_08_R_x.npy')
aaa = np.load('64_4_153_08_R_h_x.npy')
ave_power = np.load("64_4_153_08_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_08 = error_bits/(total * (loop_time-skip_time))
print(BER_D_08)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_09_R_phi_h_x.npy')
bbb= np.load('64_4_153_09_R_x.npy')
aaa = np.load('64_4_153_09_R_h_x.npy')
ave_power = np.load("64_4_153_09_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_09 = error_bits/(total * (loop_time-skip_time))
print(BER_D_09)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_1_R_phi_h_x.npy')
bbb= np.load('64_4_153_1_R_x.npy')
aaa = np.load('64_4_153_1_R_h_x.npy')
ave_power = np.load("64_4_153_1_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_detector_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_detector(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_D_10 = error_bits/(total * (loop_time-skip_time))
print(BER_D_10)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
def get_final_result_for_detector(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.5:
            r[i] = 0
    return r

def get_final_result_for_decoder(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.05:
            r[i] = 0
    return r

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_01 = error_bits/(total * (loop_time-skip_time))
print(BER_01)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_02_R_phi_h_x.npy')
bbb= np.load('64_4_153_02_R_x.npy')
aaa = np.load('64_4_153_02_R_h_x.npy')
ave_power = np.load("64_4_153_02_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_02 = error_bits/(total * (loop_time-skip_time))
print(BER_02)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_03_R_phi_h_x.npy')
bbb= np.load('64_4_153_03_R_x.npy')
aaa = np.load('64_4_153_03_R_h_x.npy')
ave_power = np.load("64_4_153_03_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_03 = error_bits/(total * (loop_time-skip_time))
print(BER_03)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_04_R_phi_h_x.npy')
bbb= np.load('64_4_153_04_R_x.npy')
aaa = np.load('64_4_153_04_R_h_x.npy')
ave_power = np.load("64_4_153_04_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_04 = error_bits/(total * (loop_time-skip_time))
print(BER_04)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_05_R_phi_h_x.npy')
bbb= np.load('64_4_153_05_R_x.npy')
aaa = np.load('64_4_153_05_R_h_x.npy')
ave_power = np.load("64_4_153_05_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_05 = error_bits/(total * (loop_time-skip_time))
print(BER_05)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_06_R_phi_h_x.npy')
bbb= np.load('64_4_153_06_R_x.npy')
aaa = np.load('64_4_153_06_R_h_x.npy')
ave_power = np.load("64_4_153_06_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_06 = error_bits/(total * (loop_time-skip_time))
print(BER_06)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_07_R_phi_h_x.npy')
bbb= np.load('64_4_153_07_R_x.npy')
aaa = np.load('64_4_153_07_R_h_x.npy')
ave_power = np.load("64_4_153_07_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_07 = error_bits/(total * (loop_time-skip_time))
print(BER_07)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_08_R_phi_h_x.npy')
bbb= np.load('64_4_153_08_R_x.npy')
aaa = np.load('64_4_153_08_R_h_x.npy')
ave_power = np.load("64_4_153_08_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_08 = error_bits/(total * (loop_time-skip_time))
print(BER_08)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_09_R_phi_h_x.npy')
bbb= np.load('64_4_153_09_R_x.npy')
aaa = np.load('64_4_153_09_R_h_x.npy')
ave_power = np.load("64_4_153_09_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_09 = error_bits/(total * (loop_time-skip_time))
print(BER_09)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_1_R_phi_h_x.npy')
bbb= np.load('64_4_153_1_R_x.npy')
aaa = np.load('64_4_153_1_R_h_x.npy')
ave_power = np.load("64_4_153_1_R_ave.npy")
v_sigma = get_sigma_by_snr(10, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_10 = error_bits/(total * (loop_time-skip_time))
print(BER_10)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
BER_01= 0.00079434705549078

In [ ]:
[BER_01,BER_02,BER_03,BER_04,BER_05]

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(40, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 1000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01 = 10 * np.log10(eve)
print(r_01)

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(8, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01_8 = 10 * np.log10(eve)
print(r_01_8)

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(6, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01_6 = 10 * np.log10(eve)
print(r_01_6)

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(12, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01_12 = 10 * np.log10(eve)
print(r_01_12)

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(14, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(bbb[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
#     result = get_final_result(output)
#     print('output first')
#     print(output)
#     print(h_x[i])
    eve += (np.power(np.linalg.norm(output[0]-aaa[i], 2), 2) / np.power(np.linalg.norm(aaa[i], 2), 2))

print(skip_time)
eve = eve / (loop_time-skip_time)
r_01_14 = 10 * np.log10(eve)
print(r_01_14)

In [ ]:
r_01

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(8, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_02_08 = error_bits/(total * (loop_time-skip_time))
print(BER_02_08)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(6, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_02_06 = error_bits/(total * (loop_time-skip_time))
print(BER_02_06)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(12, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_02_12 = error_bits/(total * (loop_time-skip_time))
print(BER_02_12)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
detector_output = np.load('64_4_153_01N_R_phi_h_x.npy')
bbb= np.load('64_4_153_01N_R_x.npy')
aaa = np.load('64_4_153_01N_R_h_x.npy')
ave_power = np.load("64_4_153_01N_R_ave.npy")
v_sigma = get_sigma_by_snr(14, ave_power, l)
error_bits = 0
error_blocks = 0
eve = 0
loop_time = 100000
skip_time = 0
output_x_o = np.ones_like(n_h_x)
# for i in range(x.shape[0]):
for i in range(loop_time):
    if (np.linalg.norm(aaa[i], 2)) == 0:
        skip_time += 1
        continue

    output = sess.run(g5_s_decoder_op, feed_dict={X1: [detector_output[i]],
                                               g_noise: np.random.normal(loc=0, scale=v_sigma, size=(1, num_input)),
                                               is_training: False})
    output_x_o[i] = output[0]
    result = get_final_result_for_decoder(output)
    temp_error = np.linalg.norm(result - bbb[i], 0)
    error_bits += temp_error
    if temp_error != 0:
        error_blocks += 1

print(skip_time)
# eve = eve / (loop_time-skip_time)
# print(10 * np.log10(eve))
print("BER=", end='')
BER_02_14 = error_bits/(total * (loop_time-skip_time))
print(BER_02_14)
print("BlockER=", end='')
print(error_blocks/(loop_time-skip_time))

In [ ]:
[r_01,r_02,r_03,r_04,r_05,r_06,r_07,r_08,r_09,r_10]

In [ ]:
[r_01_t,r_02_t,r_03_t,r_04_t,r_05_t,r_06_t,r_07_t,r_08_t,r_09_t,r_10_t]

In [ ]:
np.array([r_01,r_02,r_03,r_04,r_05,r_06,r_07,r_08,r_09,r_10]) - np.array([r_01_t,r_02_t,r_03_t,r_04_t,r_05_t,r_06_t,r_07_t,r_08_t,r_09_t,r_10_t])